# Etude sur Alien vs Predator - épisode 3

Développer un programme de vision par ordinateur capable de différencier avec précision un Alien d'un Predator.

Contexte du projet

L'épisode 2 a mis en évidence que la technique de "data augmentation" permet de diminuer de manière très efficace le surapprentissage mais n'améliore que très peu (dans notre exemple !) la performance de différenciation d'une image d'un alien d'une image d'un prédator. Dans l'épisode 3, nous allons utiliser la technique dite de "transfer learning" et plus particulièrement d'extractions de caractéristiques ("features extraction") pour améliorer les performances du programme de vision par ordinateur.

エピソード2では、データ補強技術は過学習を減らすのに非常に有効であるが、エイリアン画像と捕食者画像の区別の性能は（我々の例では！）わずかに向上するのみであることが示された。第3話では、コンピュータビジョンのプログラムの性能を向上させるために、転移学習、より具体的には特徴抽出を利用します。


La procédure à suivre est la suivante :

    Charger le modèle VGG-16 (mais qu'est-ce donc que ce VGG-16 ?) ;
    Extraire les caractéristiques des images des jeux d'apprentissage, de validation et de test via VGG-16 ;
    Entrainer un perceptron multi-couches avec le code Keras/TensorFlow suivant pour prédire à partir des caractéristiques extraites par VGG-16 la sortie (Alien ou Prédator) :

model = models.Sequential()

model.add(layers.Dense(units=256, activation='relu', input_dim=4 * 4 * 512))

model.add(layers.Dense(units=1, activation='sigmoid'))

    Conclure.

    VGG-16モデルをロードします（VGG-16とは一体何でしょうか？）
    VGG-16により、トレーニングセット、検証セット、テストセットから画像特徴を抽出する。
    VGG-16で抽出した特徴量から出力（エイリアンかプレデターか）を予測するために、以下のKeras/TensorFlowコードで多層パーセプトロンを学習させます。

model = models.Sequential()

model.add(layers.Dense(units=256, activation='relu', input_dim=4 * 4 * 512))

model.add(layers.Dense(units=1, activation=sigmoid'))

    結論を出す。




Tâches optionnelles, pour les plus courageuses et les plus courageux d'entre vous :

    Tester d'autres modèles que VGG-16 pour effectuer l'étape de "features extraction" ;
    Tester d'autres modèles de Machine Learning (par exemple SVM) pour effectuer l'étape de supervision.

オプションのタスク、勇敢なあなたへ。

    VGG-16以外のモデルで「特徴抽出」ステップをテストする。
    他の機械学習モデル（例：SVM）をテストして、監督ステップを実行する。



### 1 - import librarys

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers 
from tensorflow.keras.layers import InputLayer, Dense
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions

### 2 - set option pour l'affichage

In [2]:
pd.set_option("max_columns", None)
pd.set_option("max_colwidth", None)
pd.set_option("max_row", 500)

### 3 - charger des images

In [3]:
dataset_dir = 'Data/'

TRAINING_DIR = dataset_dir + 'train/'
VALIDATION_DIR = dataset_dir + 'validation/'
TEST_DIR = dataset_dir + 'Test/'

In [4]:
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [5]:
train_generator = datagen.flow_from_directory(TRAINING_DIR,
                                            target_size = (150, 150),                                             
                                            batch_size = 494,
                                            class_mode="binary",
                                                        #seed=0
                                              
                                             )

Found 494 images belonging to 2 classes.


In [6]:
val_generator = datagen.flow_from_directory(VALIDATION_DIR,
                                            target_size=(150, 150),
                                            batch_size = 200,
                                            class_mode="binary",
                                            #seed=0,
                                           
                                           )

Found 200 images belonging to 2 classes.


In [7]:
test_generator = datagen.flow_from_directory(TEST_DIR,
                                            target_size=(150, 150),
                                            batch_size = 200,
                                            class_mode="binary",
                                            #seed=0,                                           
                                           )

Found 200 images belonging to 2 classes.


In [8]:
X_train = train_generator[0][0]
y_train=  train_generator[0][1]

In [9]:
X_train.shape

(494, 150, 150, 3)

In [10]:
X_val = val_generator[0][0]
y_val = val_generator[0][1]

In [11]:
X_test = test_generator[0][0]
y_test = test_generator[0][1]

In [12]:
# verifiction de VGG16
model_all = VGG16(include_top=True, weights='imagenet', input_tensor=None, input_shape=None)

In [13]:
model_all.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [16]:
IMG_SHAPE = (150, 150, 3)
vgg = VGG16(input_shape = IMG_SHAPE, include_top = False,
            weights="imagenet")
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

### Feature maps VGG16 pour Train / Test / Val

In [17]:
feature_train = vgg.predict(train_generator[0][0]).reshape(494, -1)

In [18]:
feature_val = vgg.predict(val_generator[0][0]).reshape(200, -1)

In [19]:
feature_test = vgg.predict(test_generator[0][0]).reshape(200, -1)

In [20]:
feature_train.shape, feature_val.shape,  

((494, 8192), (200, 8192))

In [21]:
type(feature_train)

numpy.ndarray

In [22]:
feature_train

array([[0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       ...,
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , ..., 0.      , 4.737276, 0.      ]],
      dtype=float32)

### save features in numpy

In [40]:
np.save('feature_train', feature_train)
np.save('feature_val', feature_val)
np.save('feature_test', feature_test)
np.save('y_train', y_train)
np.save('y_val', y_val)
np.save('y_test', y_test)

preprocess_fucntion : 各入力に適用される関数。この関数は他の変更が行われる前に実行される。この関数は３次元のNumpyテンソルを引数にとり、同じshapeのテンソルを出力するように定義する必要あり。

各モデルにpreprocess_inputという便利な関数があります:モデルに適した値に正規化 ImageDataGenerator( preprocessing_function=preprocess_input, )

https://qiita.com/tatsuya11bbs/items/f2008ba8c6f127ce7524 from tensorflow.keras.applications.mobilenet_v2 import preprocess_input


https://machinelearningmastery.com/how-to-configure-image-data-augmentation-when-training-deep-learning-neural-networks/

https://github.com/tkeldenich/Computer_Vision_CNN_DataAugmentation/blob/main/Computer_Vision_CNN_DataAugmentation.ipynb